<a href="https://colab.research.google.com/github/Ilvecho/Happy-Customers/blob/main/Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install Scrapy

In [2]:
import numpy as np
import pandas as pd
import re
import scrapy
from scrapy.crawler import CrawlerProcess

from google.colab import files,drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Create the spider to crawl the desired URLs

#######
  # We want to preserve the order of the paragraphs and list items
  def combine_content(self, paragraphs, titles, list_items, link_text):
    combined_content = []

    # Add paragraphs and list items in the correct order
    while paragraphs or titles or list_items or link_text:
      if paragraphs:
        combined_content.append(paragraphs.pop(0))
      elif titles:
        combined_content.append(titles.pop(0))
      elif list_items:
        combined_content.append(list_items.pop(0))
      elif link_text:
        combined_content.append(link_text.pop(0))

    return combined_content



#################

    # Extract content from <h2> tags
    titles = response.css('h2::text').extract()

    # Extract content from <li> tags
    list_items = response.css('li::text').extract()

    # Extract the text of links in the body -> XPath //p//a/text()
    link_text = response.xpath('//p/a/text()').extract()

    # Combine paragraphs and list items in the correct order
    combined_content = self.combine_content(paragraphs, titles, list_items, link_text)

In [3]:
class HrSpider(scrapy.Spider):
  name = 'hr_spider'

  def start_requests(self):
    urls = ['https://www.getimpactly.com/post/hr-compliance-checklist',
            'https://mariopeshev.com/4-ps-employee-relations-conflict-management/']

    for url in urls:
      yield scrapy.Request(url=url, callback=self.parse)

  def parse(self, response):
    output_text = ''
    # Extract the content of interest
    paragraphs = response.css('p, li::text, h2::text').extract()

    # Stitch together everything
    for content in paragraphs:
      print(content)
      output_text = output_text + '\n' + content

    # Do the parsing
    output_text = output_text.replace("<p>","")
    output_text = output_text.replace("</p>","")
    output_text = re.sub(r'<a href="[^"]*">', '', output_text)
    output_text = output_text.replace("</a>","")

    # Save the text in a file
    with open(r'/content/gdrive/MyDrive/hr_content.txt', 'w') as text_file:
      text_file.write(output_text)
      text_file.close()


In [4]:
process = CrawlerProcess()  # Look what Scrapy settings are # settings={'FEEDS': {'item.txt': {'format': 'txt'}}}
process.crawl(HrSpider)
process.start()

INFO:scrapy.utils.log:Scrapy 2.11.0 started (bot: scrapybot)
2024-01-21 12:54:20 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.7, Platform Linux-6.1.58+-x86_64-with-glibc2.35
2024-01-21 12:54:20 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.7, Platform Linux-6.1.58+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2024-01-21 12:54:20 [scrapy.addons] INFO: Enabled addons:
[]
/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGE

The Facts
Q&A
<p>An HR compliance checklist is important for the proper functioning of an organization. HR compliance is defined as the responsibility of the business to abide by and follow the working standards set out by the employment facility. An HR compliance checklist is a systematically organized checklist that provides a list of tasks that need to be completed. Human resource specialists use it to prepare for an HR audit.</p>
What is the role of HR compliance?
<p>Compliance is required in the Human Resources department to define policies and procedures for ensuring that your employment and work practices signify a thorough understanding of all applicable laws, rules, and regulations while simultaneously also being aware of the company's larger objectives.</p>
Why is HR compliance mandatory?
<p>HR compliance is considered to be mandatory for the following reasons:</p>
To ensure that the company implements best practices for its policies
To help create manuals for hiring and reta

2024-01-21 12:54:22 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://mariopeshev.com/4-ps-employee-relations-conflict-management/> (referer: None)
INFO:scrapy.core.engine:Closing spider (finished)
2024-01-21 12:54:22 [scrapy.core.engine] INFO: Closing spider (finished)
INFO:scrapy.statscollectors:Dumping Scrapy stats:
{'downloader/request_bytes': 508,
 'downloader/request_count': 2,
 'downloader/request_method_count/GET': 2,
 'downloader/response_bytes': 70500,
 'downloader/response_count': 2,
 'downloader/response_status_count/200': 2,
 'elapsed_time_seconds': 0.90752,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2024, 1, 21, 12, 54, 22, 468892, tzinfo=datetime.timezone.utc),
 'httpcompression/response_bytes': 255510,
 'httpcompression/response_count': 2,
 'log_count/DEBUG': 3,
 'log_count/INFO': 10,
 'memusage/max': 187944960,
 'memusage/startup': 187944960,
 'response_received_count': 2,
 'scheduler/dequeued': 2,
 'scheduler/dequeued/memory': 2,
 'scheduler/

<p class="site-title"><a href="https://mariopeshev.com/" rel="home">Business Advisor</a></p>
 
 
<p>Workplace conflict costs $359 billion in paid hours or 385 million working days annually. This is according to the “<a aria-label=" (opens in a new tab)" href="https://shop.themyersbriggs.com/Pdfs/CPP_Global_Human_Capital_Report_Workplace_Conflict.pdf" target="_blank" rel="noreferrer noopener">Workplace Conflict and How Business Can Harness it to Thrive</a>” report.</p>
<p>There were news articles about <a href="https://www.forbes.com/sites/danpontefract/2021/06/05/is-apple-in-trouble-with-its-hybrid-work-model" target="_blank" rel="noreferrer noopener">Apple employees</a> having concerns about the ‘rigid’ rules in hybrid working, while Google’s people were frustrated about the ‘two tiers’ remote working.</p>
<p>No wonder why in the month of April 2021, the <a href="https://www.entrepreneur.com/article/380373" target="_blank" rel="noreferrer noopener">Great Resignation</a> started happen

In [5]:
process.stop()

<DeferredList at 0x7a8ceb082fe0 current result: []>